# 動的計画法
---
動的計画法の定義を説明することは困難だが, 抽象的な言葉を用いれば,  
> 与えられた問題全体を**一連の部分問題に分解**し, 各部分問題に対する**解をメモ化**しながら,  
> **小さな部分問題からより大きな部分問題**へと順に解を求めていく方法とされる.

動的計画法を用いれば以下のような問題を効率よく扱うことができる.  
* ナップザック問題
* スケジューリング問題  
* 発電計画問題  
* 編集距離 (diff コマンド)  
* 音声認識パターンマッチング  
* 隠れマルコフモデル  

## 例題 1. Frog問題
---
> $N$個の足場があって, $i = (0, 1, ..., N-1)$番目の足場の高さは$h_i$で与えられます.  
> 最初0番目の足場にカエルがいて, 以下のいずれかの行動を繰り返して$N-1$番目の足場を目指します.  

> * 足場iから足場$i+1$へと移動する (コストは$|h_i - h_{i+1}|$)  
> * 足場iから足場$i+2$へと移動する (コストは$|h_i - h_{i+2}|$)

> カエルがN-1番目の足場に辿り着くまでに要するコストの総和の最小値を求めてください.  
> ただし, $N=7$,$h = (2, 9, 4, 5, 1, 6, 10)$とする.  

この問題は, 以下の図のように足場を**頂点**, 移動ルートを**辺**, 移動コストを辺の**重み**としてグラフ構造化するとわかりやすくなる.  

<img src="../figs/fig_2.png">

このグラフに沿って, 頂点$i$に到達するための最小コストをメモ化することで問題を解くことが可能.  
これは, 定数時間の処理を$N$回回しているだけなので, 計算量は$O(N)$で済む.


In [20]:
import numpy as np

h = [2, 9, 4, 5, 1, 6, 10]
N = len(h)
total_cost = [None]*N

for i in range(N):
    if i == 0:
        # 0番目の足場に至るまでの最小コストをメモ化する
        total_cost[i] = 0
    if i == 1:
        # 1番目の足場に至るまでの最小コストをメモ化する
        total_cost[i] = np.abs(h[i] - h[i-1]) + total_cost[i-1]
    elif i >= 2:
        # n番目の足場に至るまでの最小コスト(2パターン)をメモを活用して計算する
        total_cost1 = np.abs(h[i] - h[i-1]) + total_cost[i-1]
        total_cost2 = np.abs(h[i] - h[i-2]) + total_cost[i-2]
        total_cost[i] = min([total_cost1, total_cost2])

total_cost

[0, 7, 2, 3, 5, 4, 8]

この問題のポイントは, **問題の全体最適解を求めるためには, その部分問題への最適化が要請される**点である.  
このような構造を, 部分構造最適性 (optimal substructure) と呼び,  
これによって, 直近の$i$番目の足場への最適ルート計算を逐次的に行うことで, $N-1$番目の足場への移動最小コストも求めることが可能である.  
このような問題の構造を利用して, 各部分問題に対する最適解を順に決定していく手法を **動的計画法 (dynamic programming: PD)** という.

# 例題2. ナップザック問題
---
> $N$個の品物があって, $i = (0, 1, ..., N-1)$番目の重さは$weight_i$ 価値は$value_i$で表されます.  
> この$N$個の品物から, 重さの総和が$W$を超えないようにいくつか品物を選びます。  
> 選んだ品物の価値の総和として考えられる最大値を求めてください。ただし, $W$や$weight$は$0$以上の整数とします.

In [55]:
def napsack(lis: list, W: int) -> int:
    # 超えてはいけない全体コストWをを各整数値w={0, 1, 2, ..., W}まで部分に分解してwを越さないように選択するというを繰り返す.
    # まず|w|の表を作成(初期化)する.
    # dp[0] はリストから"何も選んでない"ときの最大値だから, 一番目の配列は全部0にする.
    dp = [[0] * (W + 1)]
    for i in range(len(lis)): # リストの個数ぶん回す.
        print('-'*50)
        # i の周回では lis[i] までの品が選べるときの計算をします。
        value = lis[i][0]
        weight = lis[i][1]
        print(f'value: {value},  weight: {weight}')
        # lis[i] までの品が選べるときの情報を格納する配列です。もちろん [0]*W+1 です。
        dp.append([0] * (W + 1))
        print(dp)
        # それぞれの部分コストwを制約条件として, その各々について選択の是非を計算する
        for w in range(W + 1):
            print(f'w={w}')
            if weight > w:
                dp[i + 1][w] = dp[i][w]
                # 部分コストを超えてしまった場合はdp[i + 1]のw番目の配列を１つ前の結果に戻す
                continue
            # 部分コストwの制約に引っ掛からなかった場合, そのvalue
            _ = dp[i][w - weight] + value
            # lis[i] を足した場合の値と、 a[i] を足さなかったときの値(これまでの最大値)、大きいほうが、この条件下での最大値です。
            dp[i + 1][w] = max(_, dp[i][w])
            print(dp)
        print(dp)
    return (dp[-1][-1])


In [57]:
vw = [(2,3), (1, 2), (3, 6), (2, 1), (1, 3), (5, 38)]
print(napsack(vw, 4))

--------------------------------------------------
value: 2,  weight: 3
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
w=0
w=1
w=2
w=3
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 0]]
w=4
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2]]
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2]]
--------------------------------------------------
value: 1,  weight: 2
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2], [0, 0, 0, 0, 0]]
w=0
w=1
w=2
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2], [0, 0, 1, 0, 0]]
w=3
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2], [0, 0, 1, 2, 0]]
w=4
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2], [0, 0, 1, 2, 2]]
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2], [0, 0, 1, 2, 2]]
--------------------------------------------------
value: 3,  weight: 6
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2], [0, 0, 1, 2, 2], [0, 0, 0, 0, 0]]
w=0
w=1
w=2
w=3
w=4
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2], [0, 0, 1, 2, 2], [0, 0, 1, 2, 2]]
--------------------------------------------------
value: 2,  weight: 1
[[0, 0, 0, 0, 0], [0, 0, 0, 2, 2], [0, 0, 1, 2, 2], [0, 0, 1, 2, 2], [0, 0, 0, 0, 0]]
w=0
w=1
[[0, 0, 0, 0, 0], [0